In [26]:
!pip install datasets --quiet
!pip install evaluate --quiet

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [17]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
dataset = load_dataset("asparius/Persian-Food-Sentiment")

base_model = "shekar-ai/BERT-base-Persian"
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(base_model)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at shekar-ai/BERT-base-Persian and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
print(dataset['train'][:5]["text"])
print(dataset['train'][:5]["label"])

['واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح', 'قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از موقع رسید، شما ببین چقدرررررررررررر پلاک خفنههههه، من سالهاست مشتریشونم و سالهاست مزه بهشت میده غذاشون', 'قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط ظاهر فریبنده داره، پرش میکنن کالباس و قارچ', 'عالللی بود همه چه درست و به اندازه و کیفیت خوب، امیداورم همیشه کیفیتتون خوب باشه ما مشتری همیشگی بشیم', 'شیرینی وانیلی فقط یک مدل بود.']
[0, 1, 0, 1, 1]


In [18]:
from transformers import DataCollatorWithPadding
import evaluate

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

In [19]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [20]:
train_dataset = tokenized_datasets['train']
valid_dataset = tokenized_datasets['validation']
test_dataset = tokenized_datasets['test']
batch_size = 64

In [21]:
from transformers import TrainingArguments, Trainer
batch_size = 64
training_args = TrainingArguments(
    output_dir="persian_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    hub_model_id="shekar-ai/BERT-Sentiment-Persian",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.389300,0.311553,0.868286
2,0.287200,0.300371,0.876857
3,0.255100,0.316058,0.873000


TrainOutput(global_step=2658, training_loss=0.3016868287051325, metrics={'train_runtime': 659.3957, 'train_samples_per_second': 257.963, 'train_steps_per_second': 4.031, 'total_flos': 8340326510529840.0, 'train_loss': 0.3016868287051325, 'epoch': 3.0})

#Base Model - No Fine Tuning

In [25]:
from transformers import pipeline

sentiment_model = pipeline(task="text-classification", model="shekar-ai/BERT-base-Persian")
examples = ["خیلی عالی بود مرسی.", #positive
            "کیفیت غذا خیلی خیلی پایین بود متاسفم واقعا!", #negative
            " این دیگه چه غذایی بود افتضاخ بود. خیلی دیر رسید و سرد و بدمزه بود.", #negative
            "تحویل به موقع و غذای خوشمزه", #positive
            "بد بود خیلی بی کیفیت بود.",
            "غذا خوب نبود و اما محسط خیلی قشنگ و دلنشین بود."] #neutral/confusing
sentiment_model(examples)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at shekar-ai/BERT-base-Persian and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.5688220262527466},
 {'label': 'LABEL_1', 'score': 0.5167456865310669},
 {'label': 'LABEL_0', 'score': 0.5256195068359375},
 {'label': 'LABEL_1', 'score': 0.546064019203186},
 {'label': 'LABEL_1', 'score': 0.5472802519798279},
 {'label': 'LABEL_1', 'score': 0.5069504976272583}]

#Finetuned Model

In [24]:
from transformers import pipeline

sentiment_model = pipeline(task="text-classification", model="shekar-ai/BERT-Sentiment-Persian")
examples = ["خیلی عالی بود مرسی.", #positive
            "کیفیت غذا خیلی خیلی پایین بود متاسفم واقعا!", #negative
            " این دیگه چه غذایی بود افتضاخ بود. خیلی دیر رسید و سرد و بدمزه بود.", #negative
            "تحویل به موقع و غذای خوشمزه", #positive
            "بد بود خیلی بی کیفیت بود.",
            "غذا خوب نبود و اما محسط خیلی قشنگ و دلنشین بود."] #neutral/confusing
sentiment_model(examples)

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9969315528869629},
 {'label': 'NEGATIVE', 'score': 0.9943053126335144},
 {'label': 'NEGATIVE', 'score': 0.9880203604698181},
 {'label': 'POSITIVE', 'score': 0.9965628981590271},
 {'label': 'NEGATIVE', 'score': 0.9898874163627625},
 {'label': 'POSITIVE', 'score': 0.5858227610588074}]